## Mnist dataset download and parsing

- 참조 : https://m.blog.naver.com/PostView.nhn?blogId=acwboy&logNo=220584307823&proxyReferer=https%3A%2F%2Fwww.google.com%2F


In [52]:
import requests
from bs4 import BeautifulSoup
import os
import gzip
import numpy as np
import matplotlib.pyplot as plt
from struct import *
import scipy.misc.imsave as imsave

def example1(mnist_url):
    
    mnist_url = 'http://yann.lecun.com/exdb/mnist/'
    save_folder = './mnist_download'

    if os.path.exists(save_folder) != True : os.mkdir(save_folder)

    page = requests.get(mnist_url)
    soup = BeautifulSoup(page.content, 'html.parser')

    # download part
    for a_tag in soup.find_all('a'):
        if '.gz' in str(a_tag):
            print(a_tag.text)
            a = requests.get(mnist_url + a_tag.text, allow_redirects = True)

            with open(os.path.join(save_folder, a_tag.text), 'wb') as f:

                f.write(a.content)

                f.close()

    # Parsing and save

    train_dir = '../data/mnist_dataset/train'
    test_dir = '../data/mnist_dataset/test'

    if not os.path.exists(train_dir): os.mkdir(train_dir)
    if not os.path.exists(test_dir): os.mkdir(test_dir)

    train_image_file = 'train-images-idx3-ubyte.gz'
    train_label_file = 'train-labels-idx1-ubyte.gz'
    test_image_file = 't10k-images-idx3-ubyte.gz'
    test_label_file = 't10k-labels-idx1-ubyte.gz'
    
    

    def mnistByte2Imgfolder(img_byte_file, label_byte_file, save_dir):

        img_byte = gzip.open(img_byte_file , 'rb')
        label_byte = gzip.open(label_byte_file , 'rb')

        img_byte.read(16)
        label_byte.read(8)

        while True:

            img = img_byte.read(784)
            label = label_byte.read(1)

            if not img or not label : break

            idx = int(label[0])
            img = np.reshape( unpack(len(img)*'B',img), (28,28))

            label_dir = os.path.join(save_dir, str(idx))
            if not os.path.exists(label_dir) : 
                os.mkdir(label_dir)
                if os.path.exists(os.path.join(label_dir, '.DS_Store')):
                    os.remove(os.path.join(label_dir, '.DS_Store')) # mac 의 경우

            file_index = str(len(os.listdir(label_dir)) + 1)

            imsave(os.path.join(label_dir, file_index+'.jpg'), img)

        return None

    
    mnistByte2Imgfolder(os.path.join(train_image_file), os.path.join(train_label_file), train_dir)
    mnistByte2Imgfolder(os.path.join(test_image_file), os.path.join(test_label_file), test_dir)
    
    print('Mnist dataset Ready')
    return None

train-images-idx3-ubyte.gz
train-labels-idx1-ubyte.gz
t10k-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte.gz
